In [18]:
# ライブラリをインポート

# データ操作と数値計算のため
import pandas as pd
import numpy as np

# データ可視化のため
import matplotlib.pyplot as plt
import seaborn as sns

# 統計モデリングと計量経済分析のため
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from statsmodels.iolib.summary2 import summary_col

# for ignore warning
import warnings
warnings.filterwarnings('ignore')

In [31]:
url = "https://www.fbc.keio.ac.jp/~tyabu/keiryo/protection_data.csv"
df = pd.read_csv(url)
df = df.dropna()
df.head()

,country,protection,gdp1995,mortality,latitude,europop
0,AGO,5.363636,7.770645,5.634789,0.136667,8.0
1,ARG,6.386364,9.133459,4.232656,0.377778,60.0
2,AUS,9.318182,9.897972,2.145931,0.300000,98.0
3,BFA,4.454545,6.845880,5.634789,0.144444,0.0
4,BGD,5.136364,6.877296,4.268438,0.266667,0.0


## 13.6節の推定結果

### OLS

In [58]:
X = sm.add_constant(df['protection'])
y = df['gdp1995']
out_1 = sm.OLS(y, X).fit(cov_type='HC1')
print(out_1.summary())

                            OLS Regression Results                            
Dep. Variable:                gdp1995   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                     105.3
Date:                Fri, 30 Aug 2024   Prob (F-statistic):           6.52e-15
Time:                        17:09:26   Log-Likelihood:                -66.578
No. Observations:                  63   AIC:                             137.2
Df Residuals:                      61   BIC:                             141.4
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6791      0.322     14.513      0.0

### IV

#### First-stage

In [62]:
X = sm.add_constant(df[['mortality']])
y = df['protection']
out_2_1st = sm.OLS(y, X).fit(cov_type='HC1')
print(out_2_1st.summary())

                            OLS Regression Results                            
Dep. Variable:             protection   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.257
Method:                 Least Squares   F-statistic:                     15.75
Date:                Fri, 30 Aug 2024   Prob (F-statistic):           0.000193
Time:                        17:10:41   Log-Likelihood:                -103.63
No. Observations:                  63   AIC:                             211.3
Df Residuals:                      61   BIC:                             215.5
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.3869      0.732     12.817      0.0

In [60]:
x = sm.add_constant(df['protection'])
y = df['gdp1995']
w = None
Z = sm.add_constant(df['mortality'])
out_2 = IV2SLS(y, w, X, Z).fit()
print(out_2)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                gdp1995   R-squared:                      0.2087
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1958
No. Observations:                  63   F-statistic:                    28.636
Date:                Fri, Aug 30 2024   P-value (F-stat)                0.0000
Time:                        17:09:37   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          2.0431     1.1479     1.7798     0.0751     -0.2068      4.2929
protection     0.9221     0.1723     5.3512     0.00

## 13.7節の推定結果

### First-stage

In [63]:
X = sm.add_constant(df[['latitude', 'mortality', 'europop']])
y = df['protection']
out_3_1st = sm.OLS(y, X).fit(cov_type='HC0')
print(out_3_1st.summary())

                            OLS Regression Results                            
Dep. Variable:             protection   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     17.57
Date:                Fri, 30 Aug 2024   Prob (F-statistic):           2.87e-08
Time:                        17:10:54   Log-Likelihood:                -99.061
No. Observations:                  63   AIC:                             206.1
Df Residuals:                      59   BIC:                             214.7
Df Model:                           3                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.8531      0.970      8.093      0.0

### Second-stage

In [34]:
x = sm.add_constant(df['protection'])
y = df['gdp1995']
w = df['latitude']
Z = sm.add_constant(df[['mortality', 'europop']])
out_3 = IV2SLS(y, w, X, Z).fit()
print(out_3)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                gdp1995   R-squared:                      0.1742
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1467
No. Observations:                  63   F-statistic:                    39.784
Date:                Fri, Aug 30 2024   P-value (F-stat)                0.0000
Time:                        16:56:46   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
latitude      -0.5971     0.9505    -0.6282     0.5298     -2.4600      1.2658
const          1.9947     1.0400     1.9181     0.05

### F検定

In [37]:
X = sm.add_constant(df[['mortality', 'europop', 'latitude']])
y = df['protection']
out_4 = sm.OLS(y, X).fit()
print(out_4.summary())

                            OLS Regression Results                            
Dep. Variable:             protection   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     11.42
Date:                Fri, 30 Aug 2024   Prob (F-statistic):           5.25e-06
Time:                        16:57:41   Log-Likelihood:                -99.061
No. Observations:                  63   AIC:                             206.1
Df Residuals:                      59   BIC:                             214.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.8531      0.831      9.452      0.0

In [50]:
f_test = out_4.f_test('mortality = europop = 0')
print(f"F-statistic: {f_test.fvalue:.2f}")
print(f"p-value: {f_test.pvalue:.4f}")

F-statistic: 10.52
p-value: 0.0001


#### 過剰識別検定
13.7.3節の過剰識別性検定を行う。

p値が10%を上回れば、帰無仮説が採択される(操作変数の外生性が満たされる)。

p値が10%を下回れば、帰無仮説が棄却される(操作変数の外生性が満たされない)。


In [66]:
exog = sm.add_constant(df['latitude'])
endog = df['protection']
instruments = df[['mortality', 'europop']]
out_5 = IV2SLS(df['gdp1995'], exog, endog, instruments).fit()

print(out_5.wooldridge_regression)
print(f'\n{out_5.wooldridge_overid}')

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 33.4047
P-value: 0.0000
Distributed: chi2(1)

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 0.0668
P-value: 0.7961
Distributed: chi2(1)
